In [1]:
# You only need to run this once when you load the notebook to install required packages. You can comment this cell out once you run it.

# !pip install torch
!pip install datasets
!pip install apache_beam mwparserfromhell
!pip install transformers[torch]
!pip install sentence_transformers
!pip install evaluate
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 6.4 MB/s eta 0:00:00
  Preparing met

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
import os
import pandas as pd

file_path = "/content/drive/MyDrive/Synthesizing Digital Efforts/Chatbot/billboardHot100_1999-2019.csv"
df = pd.read_csv(file_path)

# Get the top 10 artists
top_artists = df['Artists'].value_counts().head(10).index

# Create a DataFrame to store combined lyrics
combined_lyrics_df = pd.DataFrame(columns=['Artists', 'combined_lyrics'])

# Combine lyrics for each artist
for artist in top_artists:
    artist_lyrics = df[df['Artists'] == artist]['Lyrics'].dropna().tolist()
    combined_lyrics = '\n'.join(artist_lyrics)

    # Append to the combined_lyrics_df
    combined_lyrics_df = combined_lyrics_df.append({'Artists': artist, 'combined_lyrics': combined_lyrics}, ignore_index=True)


print("Combined lyrics dataset for the top 10 artists has been created.")



Combined lyrics dataset for the top 10 artists has been created.


<ipython-input-4-3c70b51aa195>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_lyrics_df = combined_lyrics_df.append({'Artists': artist, 'combined_lyrics': combined_lyrics}, ignore_index=True)
<ipython-input-4-3c70b51aa195>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_lyrics_df = combined_lyrics_df.append({'Artists': artist, 'combined_lyrics': combined_lyrics}, ignore_index=True)
<ipython-input-4-3c70b51aa195>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_lyrics_df = combined_lyrics_df.append({'Artists': artist, 'combined_lyrics': combined_lyrics}, ignore_index=True)
<ipython-input-4-3c70b51aa195>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

In [5]:
print(combined_lyrics_df['Artists'])

0            Drake
1     Taylor Swift
2          Rihanna
3    Kenny Chesney
4         Maroon 5
5      Keith Urban
6          Beyonce
7      Chris Brown
8       Tim McGraw
9    Rascal Flatts
Name: Artists, dtype: object


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Check the total number of rows in the dataset
total_rows = len(df)

# Define the percentage for splitting
train_percentage = 0.8  # 80% for training, 20% for testing

# Ensure the dataset has at least 10 rows
if total_rows < 10:
    raise ValueError("The dataset has fewer than 10 rows.")

# Split the dataset into training and test sets
train_df, test_df = train_test_split(combined_lyrics_df, train_size=train_percentage, random_state=42)

print("Dataset split into training and test sets.")


Dataset split into training and test sets.


In [7]:
context_len = 512

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

#Tokenization using GPT2, with the intention of upgrading in the future as we increase complexity.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # Alternatively, other special tokens could be used. These tokens are just used to pad the context up to the context length incase your text is short.

def tokenize_function(examples):
    return tokenizer(examples['combined_lyrics'], truncation=True, max_length=context_len, padding='max_length')

train_dataset_tokenized = train_dataset.map(tokenize_function)
test_dataset_tokenized = test_dataset.map(tokenize_function)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [16]:

test_dataset_tokenized

Dataset({
    features: ['Artists', 'combined_lyrics', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2
})

In [17]:
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

In [18]:
#training

training_args = TrainingArguments(
    per_device_train_batch_size=2, # Setting the batch size low helps with memory issues.
    per_device_eval_batch_size=2, # Setting the batch size low helps with memory issues.
    logging_dir='./pre_trained/logs',
    evaluation_strategy="epoch", # Setting this to "epoch" instead of "step" speeds up the training because evaluations are not made for every batch.
    logging_strategy="epoch", # Setting this to "epoch" instead of "step" speeds up the training because logging is not made for every batch.
    save_strategy="epoch", # Model saving happens at the epoch level, which is more efficient than at the batch level.
    save_total_limit=1,  # Only save the best model
    output_dir="./pre_trained/results",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    # learning_rate=5e-4,
    num_train_epochs=10, # Sets number of epochs to 10.
    load_best_model_at_end=True, # IMPORTANT: this is what loads the best model at the end of training.
    metric_for_best_model="eval_loss", # Optional as "eval_loss" is the default value, but emphasizes that we care about best on "eval" set, not train set.
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # Handles padding the different texts to make them all same lengths and batchable.

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    # compute_metrics=compute_metrics  # This line calls compute_metrics to compute the perplexity per epoch. However, in case you are having memory issues, it could be commented out for better efficiency.
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.754900,2.208123
2,2.500900,2.206009
3,2.373300,2.208241
4,2.262600,2.210763
5,2.170500,2.219878
6,2.121800,2.227581
7,2.051000,2.234260
8,1.997100,2.241447
9,1.969400,2.244706
10,1.956200,2.245932


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=40, training_loss=2.2157716155052185, metrics={'train_runtime': 125.2283, 'train_samples_per_second': 0.639, 'train_steps_per_second': 0.319, 'total_flos': 20903362560000.0, 'train_loss': 2.2157716155052185, 'epoch': 10.0})

In [19]:
saved_model_name = "/content/drive/MyDrive/Synthesizing Digital Efforts/Chatbot/lyrics_best_model"

trainer.save_model(saved_model_name)

In [20]:
best_model = GPT2LMHeadModel.from_pretrained(saved_model_name).to(device)

In [21]:
def gen_text(model, artist, temperature=1.0, max_gen_tokens=40, prefix=tokenizer.bos_token):

    # Tokenize input text
    inputs = tokenizer(prefix, return_tensors="pt", truncation=True, max_length=context_len, return_special_tokens_mask=True)

    # Move inputs to GPU
    inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

    # Generate text using the best model
    output_ids = model.generate(inputs["input_ids"],
                                num_return_sequences=5,
                                attention_mask=inputs["attention_mask"],
                                max_new_tokens=max_gen_tokens,
                                temperature=temperature,
                                pad_token_id=tokenizer.pad_token_id,
                                do_sample=True)

    # Print the generated sentences
    for i in range(5):
        print('----')

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[i].tolist(), skip_special_tokens=True)

        print(generated_text)

# Example usage
selected_artist = "Taylor Swift"  # Replace with the artist you want to generate lyrics for
prefix = "Style"  # Replace with the desired starting lyrics
gen_text(best_model, selected_artist, temperature=0.4, max_gen_tokens=100, prefix=prefix)


----
Style, Textures, and Textures

A new version of the Unity Editor is available for download. It is a standalone editor for Unity. It is designed to run on any Unity system. It is compatible with all Unity game engines and includes a number of new features.

Unity Editor Features

Unity Editor Features

Unity Editor Features

Unity Editor Features

Unity Editor Features

Unity Editor Features

Unity Editor Features

Unity Editor Features

Unity Editor Features
----
Style.com/forum/showthread.php?t=183588

I've been working on a new version of this plugin for a while now. I've been using it for about a year now, and I've been using it for about a year now. I've been using it for about a year now, and I've been using it for about a year now. I've been using it for about a year now, and I've been using it for about a year
----
Style.com/item/847-3-4-9-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-
----
Style.java:8]

[08/17/2015 - 12:38:22PM] Err

In [22]:
def gen_text(model, artist, temperature=1.0, max_gen_tokens=40, prefix=tokenizer.bos_token, context_tokens=None):

    # Tokenize input text
    inputs = tokenizer(prefix + context_tokens, return_tensors="pt", truncation=True, max_length=context_len, return_special_tokens_mask=True)

    # Move inputs to GPU
    inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

    # Generate text using the best model
    output_ids = model.generate(inputs["input_ids"],
                                num_return_sequences=5,
                                attention_mask=inputs["attention_mask"],
                                max_new_tokens=max_gen_tokens,
                                temperature=temperature,
                                pad_token_id=tokenizer.pad_token_id,
                                do_sample=True)

    # Print the generated sentences
    generated_lyrics = []
    for i in range(5):
        print('----')

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[i].tolist(), skip_special_tokens=True)

        print(generated_text)
        generated_lyrics.append(generated_text)

    return generated_lyrics

# Example usage with additional context tokens
selected_artist = "Taylor Swift"
prefix = "You've been hurting"
context_tokens = " and I have been waiting"
generated_lyrics = gen_text(best_model, selected_artist, temperature=0.8, max_gen_tokens=100, prefix=prefix, context_tokens=context_tokens)

# Combine multiple outputs
combined_output = "\n".join(generated_lyrics)
print("Generated Lyrics:")
print(combined_output)


----
You've been hurting and I have been waiting for you to come here. So, you will come here to change my life forever. I will be here with you forever."

The two were accompanied by a large group of people who shouted at them to leave.

"Come on, stay with the family for this. I'm sorry to disappoint everybody, but if you don't go, there's no one to take care of you."

An older man spoke, "My name is J.R. Taylor. I
----
You've been hurting and I have been waiting for you to start feeling better about yourself. I hope you're all in the right place. Thank you all.

It sounds like you've finally found your sweet spot, and I'd like to make sure that you're in the right place. I want you to be happy. I want you to be so completely on your own. You know, not alone. Not yet. Not yet. You deserve all of it. You deserve to have a new life, a new love. You
----
You've been hurting and I have been waiting for this moment to come all the way to my hands," she said.

According to the U.S. Attorne